In [1]:
pip install psycopg2-binary


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xlsxwriter


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [151]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install requests


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import psycopg2
import pandas as pd
import numpy as np
import requests

In [31]:
#Conexión Lake Cencommerce
# Update the following with your Redshift connection details
redshift_host = 'rs-cl-sbx.csmzzf6sb3pd.us-east-1.redshift.amazonaws.com'
redshift_ip = '172.23.55.223'
redshift_port = '5439'
redshift_db = 'sandbox_corp'
redshift_user = 'elias_salehmuse'
redshift_password = 'vtMPcp2u3B3q'

def execute_query(query):
    try:
        # Establish a connection to Redshift
        conn = psycopg2.connect(
            host=redshift_host,
            port=redshift_port,
            database=redshift_db,
            user=redshift_user,
            password=redshift_password
        )

        # Create a cursor
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        result = cursor.fetchall()

        # Get column names
        columns = [desc[0] for desc in cursor.description]

        # Create a Pandas DataFrame
        df = pd.DataFrame(result, columns=columns)

        # Close the cursor and connection
        cursor.close()
        conn.close()

        return df

    except Exception as e:
        print(f"Error: {e}")

In [32]:
#Conexión BDD AUX Delivery
# Update the following with your Redshift connection details
redshift_host1 = 'daas-report-bagging.crrrlku8fkeu.us-east-1.rds.amazonaws.com'
redshift_ip1= '172.23.55.223'
redshift_port1 = '5432'
redshift_db1 = 'daas_report_bagging'
redshift_user1 = 'elias_saleh_db'
redshift_password1 = 'yCWkGK9H7qO6JB6LLhOu3'

def execute_query1(query1):
    try:
        # Establish a connection to Redshift
        conn1= psycopg2.connect(
            host=redshift_host1,
            port=redshift_port1,
            database=redshift_db1,
            user=redshift_user1,
            password=redshift_password1
        )

        # Create a cursor
        cursor1 = conn1.cursor()

        # Execute the query
        cursor1.execute(query1)

        # Fetch all results
        result1 = cursor1.fetchall()

        # Get column names
        columns1 = [desc[0] for desc in cursor1.description]

        # Create a Pandas DataFrame
        df1 = pd.DataFrame(result1, columns=columns1)

        # Close the cursor and connection
        cursor1.close()
        conn1.close()

        return df1

    except Exception as e:
        print(f"Error: {e}")

In [68]:
#Query Filters

fecha_compromiso_inicial = '2024-02-01 0:00:00'
fecha_compromiso_final = '2024-02-29 23:59:59'
negocio = 'Jumbo' #Jumbo, Santa Isabel, Spid (Chile)


In [69]:
# Example query lake
query = f"""
select *
from (
select dispatch_order.external_system_order_id as orden_compra, 

hd_delivery_dispatch_raw_prod_spec.courier.name as sistema,
hd_delivery_dispatch_raw_prod.dispatch_order.external_driver_id as codigo_conductor,

case
when hd_delivery_dispatch_raw_prod_spec.courier.name='Beetrack' then tags1.value
when hd_delivery_dispatch_raw_prod_spec.courier.name='Home Delivery Chile' then provider.name
else hd_delivery_dispatch_raw_prod_spec.courier.name
end
as proveedor,

case
when hd_delivery_dispatch_raw_prod_spec.courier.name='Beetrack' then tags2.value
when hd_delivery_dispatch_raw_prod_spec.courier.name='Home Delivery Chile' then hd_delivery_tms_raw_prod_spec.vehicle.licence_plate
else hd_delivery_dispatch_raw_prod_spec.courier.name
end
as patente,
case
when hd_delivery_dispatch_raw_prod_spec.courier.name='Beetrack' then hd_delivery_dispatch_raw_prod.vehicle_driver.full_name
else hd_delivery_tms_raw_prod_spec.driver.full_name
end
as nombre_driver,


retail_chain.name as negocio,
known_source.external_id as codigo_tienda, 
known_source.name as nombre_tienda,





tmp_te."event-date"::timestamp as fecha_evento,




case
when hd_delivery_dispatch_raw_prod_spec.country.code='PE' then
date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-5')
when hd_delivery_dispatch_raw_prod_spec.country.code='CO' then
date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-5')
when hd_delivery_dispatch_raw_prod_spec.country.code='AR' then
date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-3')
when hd_delivery_dispatch_raw_prod_spec.country.code='BR' then
date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-3')
else date(dispatch_order.eta_end_date::timestamp)
end
as fecha_compromiso,

pack2.last_status_desc as nombre_estado, 
pack2.last_sub_status_desc as nombre_sub_estado, 





saz2_destination.name as comuna_destino,

destination.latitude as latitude,
destination.longitude as longitude,

case
when hd_delivery_dispatch_raw_prod_spec.country.code='PE' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp - INTERVAL '1 hour' )<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-5')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') < '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='PE' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp - INTERVAL '2 hour' )<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-5')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') >= '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='CO' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp - INTERVAL '1 hour' )<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-5')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') < '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='CO' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp - INTERVAL '2 hour' )<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-5')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') >= '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='AR' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp + INTERVAL '1 hour' )<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-3')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') < '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='AR' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp)<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-3')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') >= '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='BR' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp + INTERVAL '1 hour' )<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-3')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') < '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='BR' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp)<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-3')) and date(dispatch_order.eta_end_date::timestamp AT TIME ZONE 'UTC-4') >= '09/03/2023'then
1
when hd_delivery_dispatch_raw_prod_spec.country.code='CL' and (DATE_TRUNC('minute', tmp_te."event-date"::timestamp)<= DATE_TRUNC('minute',dispatch_order.eta_end_date::timestamp))then
1
else 0
end
as on_time,
--tags_bultos.total::integer as total_bultos,
--tags_bultos.size as size,
dispatch_order.external_system_order_id ||'|'|| max(dispatch_order.updated_at::timestamp) as order_key



from hd_delivery_dispatch_raw_prod.dispatch_order

--Pack

left join 
(select *
from (
select
pack.dispatch_order_id,pack.tracking, pack.last_status_code, pack.last_status_desc,
pack.last_sub_status_code,pack.last_sub_status_desc,  pack.tracking ||'|'|| max(pack.updated_at::timestamp) as pack_key

from hd_delivery_dispatch_raw_prod.pack 

--where pack.tracking = 'v500140173spid-01'

group by 1,2,3,4,5,6
) as tempo
where tempo.pack_key in (select tmp00.pack_key
from(
select pack.tracking, pack.tracking ||'|'|| max(pack.updated_at::timestamp) as pack_key
from hd_delivery_dispatch_raw_prod.pack 
--where pack.tracking = 'v500140173spid-01'
group by 1) tmp00)



) as pack2



on pack2.dispatch_order_id= dispatch_order.id

--Destination
left join hd_delivery_dispatch_raw_prod.destination on destination.id = dispatch_order.destination_id
left join hd_delivery_dispatch_raw_prod_spec.sub_admin_zone_1 saz1_destination on saz1_destination.id = destination.sub_admin_zone_1_id
left join hd_delivery_dispatch_raw_prod_spec.sub_admin_zone_2 saz2_destination on saz2_destination.id = destination.sub_admin_zone_2_id
--Known Source
left join hd_delivery_dispatch_raw_prod_spec.known_source on known_source.id=dispatch_order.owner_known_source_id

-- Retail chain
left join hd_delivery_dispatch_raw_prod_spec.retail_chain on hd_delivery_dispatch_raw_prod_spec.retail_chain.id=hd_delivery_dispatch_raw_prod.dispatch_order.retail_chain_id
-- Country
left join hd_delivery_dispatch_raw_prod_spec.country on hd_delivery_dispatch_raw_prod_spec.country.id=hd_delivery_dispatch_raw_prod_spec.retail_chain.country_id
-- Dispatch Type
left join hd_delivery_dispatch_raw_prod_spec.dispatch_type on hd_delivery_dispatch_raw_prod.dispatch_order.dispatch_type_id = hd_delivery_dispatch_raw_prod_spec.dispatch_type.id
-- Courier
left join hd_delivery_dispatch_raw_prod_spec.courier on hd_delivery_dispatch_raw_prod_spec.courier.id = hd_delivery_dispatch_raw_prod.dispatch_order.courier_id
-- Driver TMS
left join hd_delivery_tms_raw_prod_spec.driver on hd_delivery_tms_raw_prod_spec.driver.id = hd_delivery_dispatch_raw_prod.dispatch_order.external_driver_id
-- Vehicle
left join hd_delivery_tms_raw_prod_spec.vehicle on hd_delivery_tms_raw_prod_spec.vehicle.id= hd_delivery_tms_raw_prod_spec.driver.vehicle_id

-- Provider 
left join hd_delivery_dispatch_raw_prod_spec.provider on hd_delivery_tms_raw_prod_spec.driver.provider_id = hd_delivery_dispatch_raw_prod_spec.provider.id
-- Event Date
left join 
(select te.external_system_order_id, te."event-date", ps.description

from hd_delivery_tracking_raw_prod.tracking_event te

left join hd_delivery_tracking_raw_prod_spec.pack_status ps on te.pack_status_id= ps.id

where te.id in 
(select max(te0.id) as id0
from hd_delivery_tracking_raw_prod.tracking_event te0
group by te0.external_system_order_id)) as tmp_te
on dispatch_order.external_system_order_id = tmp_te.external_system_order_id

-- Bultos
--left join 
--(select *,
--case
--when tmp_tags.total>0 and tmp_tags.total<=5 then 'MP'
--when tmp_tags.total>5 and tmp_tags.total<=12 then 'P'
--when tmp_tags.total>12 and tmp_tags.total<=20 then 'M'
--when tmp_tags.total>20 then 'G'
--else ''
--end 
--as size

--from (
--select
--id,
--created_at,
--updated_at,
--name,
--value,
--dispatch_order_id,


--REGEXP_COUNT(value, '"quantity":1')+REGEXP_COUNT(value, '"quantity":2')*2+REGEXP_COUNT(value, '"quantity":3')*3+REGEXP_COUNT(value, '"quantity":4')*4+REGEXP_COUNT(value, '"quantity":5')*5+
--REGEXP_COUNT(value, '"quantity":6')*6+REGEXP_COUNT(value, '"quantity":7')*7+REGEXP_COUNT(value, '"quantity":8')*8+REGEXP_COUNT(value, '"quantity":9')*9+REGEXP_COUNT(value, '"quantity":10')*10+
--REGEXP_COUNT(value, '"quantity":11')*11+REGEXP_COUNT(value, '"quantity":12')*12+REGEXP_COUNT(value, '"quantity":13')*13+REGEXP_COUNT(value, '"quantity":14')*14+REGEXP_COUNT(value, '"quantity":15')*15
--as total
--from
--hd_delivery_dispatch_raw_prod."tag"
--where 1=1
--and hd_delivery_dispatch_raw_prod."tag".name='items'
--) as tmp_tags
--) as tags_bultos
--on dispatch_order.id = tags_bultos.dispatch_order_id

left join
(
select dispatch_order_id,name, value, id

from hd_delivery_dispatch_raw_prod."tag"
where 1=1 
and (lower(hd_delivery_dispatch_raw_prod."tag".name) like '%transportes%')
--and dispatch_order_id = 15258246
--group by dispatch_order_id,name,value
and hd_delivery_dispatch_raw_prod."tag".id in (
select aux0.maxid
from(
select
dispatch_order_id,name,max (id) as maxid
from hd_delivery_dispatch_raw_prod."tag"
where 1=1
--and dispatch_order_id = 14496340
and (lower(hd_delivery_dispatch_raw_prod."tag".name) like '%transportes%')
group by dispatch_order_id,name)as aux0
)) as tags1
on dispatch_order.id = tags1.dispatch_order_id



left join
(
select dispatch_order_id,name, value, id

from hd_delivery_dispatch_raw_prod."tag"
where 1=1 
and (hd_delivery_dispatch_raw_prod."tag".name='truck_identifier'
)
--and dispatch_order_id = 15258246
--group by dispatch_order_id,name,value
and hd_delivery_dispatch_raw_prod."tag".id in (
select aux1.maxid
from(
select
dispatch_order_id,name,max (id) as maxid
from hd_delivery_dispatch_raw_prod."tag"
where 1=1
--and dispatch_order_id = 14496340
and (hd_delivery_dispatch_raw_prod."tag".name='truck_identifier'
)
group by dispatch_order_id,name)as aux1
)) as tags2
on dispatch_order.id = tags2.dispatch_order_id






left join hd_delivery_dispatch_raw_prod.vehicle_driver on hd_delivery_dispatch_raw_prod.vehicle_driver.id=hd_delivery_dispatch_raw_prod.dispatch_order.vehicle_driver_id



where 1=1
and dispatch_order.retail_chain_id in (1,4,9)
and pack2.last_status_code not in ('10','1002')
and pack2.last_status_code in ('09','02','04','03')
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17


--[[and {{created_at}}]]
--[[and {{update_at}}]]
--[[and {{nombre_tienda}}]]
--[[and {{codigo_tienda}}]]
--[[and {{orden_compra}}]]
--[[and {{estado}}]]
--[[and {{subestado}}]]
--[[and {{patente}}]]
--[[and {{nombre_conductor}}]]
--[[and {{negocio}}]]
--[[and {{courier}}]]
) as tmp
where 1=1
and tmp.order_key in (select tmp000.order_key
from(
select dispatch_order.external_system_order_id, dispatch_order.external_system_order_id ||'|'|| max(dispatch_order.updated_at::timestamp) as order_key
from hd_delivery_dispatch_raw_prod.dispatch_order
--where dispatch_order.external_system_order_id = 'v500140173spid-01'
group by 1) as tmp000)


and tmp.negocio = '{negocio}'
and tmp.proveedor <> 'None'
and tmp.nombre_driver <> 'None'
--and tmp.fecha_evento >= '2023-12-01 0:00:00'
and tmp.fecha_compromiso >= '{fecha_compromiso_inicial}'
and tmp.fecha_compromiso <= '{fecha_compromiso_final}'



--[[and  tmp.fecha_evento>= ({{updated_at1}}::date || ' 0:00:00')::timestamp]]
--[[and  tmp.fecha_evento<= ({{updated_at2}}::date || ' 23:59:59')::timestamp]]
--[[and  tmp.fecha_compromiso>= ({{eta_1}}::date || ' 0:00:00')::timestamp]]
--[[and  tmp.fecha_compromiso<= ({{eta_2}}::date || ' 23:59:59')::timestamp]]
--[[and tmp.proveedor = {{proveedor}}]]




"""



In [70]:
#Query Aux

query1= f"""
select order_id,
sum(case 
	when ROUND(quantity::numeric, 0) = quantity THEN quantity::integer
    ELSE 1
end) as bultos

from daas_report_bagging.report_bagging.item
group by order_id
"""

**EJECUTAR 1**

In [71]:
# Execute the query and get the result in a DataFrame
result_df = execute_query(query)


**EJECUTAR 2**

In [72]:
result_df1 = execute_query1(query1)

In [ ]:
#Merges with another data sources

**EJECUTAR 3**

In [73]:
# Display the DataFrame

result_df

#System Filter
#result_df_1= result_df[result_df['sistema']=='Home Delivery Chile']

#Provider Filter
#result_df_2= result_df[result_df['proveedor']=='Boosmap Chile']
#result_df_2

#Store Code Filter
#result_df_3= result_df[result_df['codigo_tienda']=='J762']
#result_df_3



,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key
0,v122867410jmch-01,Beetrack,,Touch,TOUCH-HVPB33,Oscar Henriquez Fierro,Jumbo,J502,JUMBO KENNEDY,2024-02-10 19:25:51,2024-02-10,Entregado,Entregado a Domicilio,Las Condes,-33.399614800000000,-70.569115500000000,1,v122867410jmch-01|2024-02-10 19:27:17.499
1,v123879936jmch-01,Beetrack,,Foxer,FOXER-PSZB37,José Lara Carrizales,Jumbo,J591,JUMBO TALCA,2024-02-24 09:52:00,2024-02-24,Entregado,Entregado a Domicilio,Talca,-35.428734576275600,-71.615340886046900,1,v123879936jmch-01|2024-02-24 09:53:09.134
2,v122397379jmch-01,Beetrack,,ZUBALE,ZUBALE-SBRC47,Ysmar Anselmi,Jumbo,J989,JUMBO INDEPENDENCIA,2024-02-04 15:40:13,2024-02-04,Entregado,Entregado a Domicilio,Santiago,-33.432082500000000,-70.659157700000000,1,v122397379jmch-01|2024-02-04 15:41:33.359
3,v123190357jmch-01,Beetrack,,Boosmap,BOOS-UG2493,CLAUDIO ANTONIO VERGARA CHAVEZ,Jumbo,J403,JUMBO MAIPU,2024-02-15 09:54:44,2024-02-15,Entregado,Entregado a Domicilio,Maipú,-33.523223700000000,-70.772186800000000,1,v123190357jmch-01|2024-02-15 09:56:02.361
4,v122704818jmch-01,Beetrack,,Boosmap,BOOS-VN5369,Carolina Eugenia Aguilera Rojas,Jumbo,J843,JUMBO SAN BERNARDO,2024-02-08 18:14:09,2024-02-08,Entregado,Entregado a Domicilio,San Bernardo,-33.586949300000000,-70.697080000000000,1,v122704818jmch-01|2024-02-08 18:15:23.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474746,v123363848jmch-01,Beetrack,,Transporte CM,TCM-YZ7148,DIEGO ANDRES VILLOUTA CERDA,Jumbo,J633,JUMBO CONCE. HUALPEN,2024-02-17 13:48:04,2024-02-17,Entregado,Entregado a Domicilio,Concepción,-36.803690000000000,-73.045779900000000,1,v123363848jmch-01|2024-02-17 13:49:15.969
474747,v122653017jmch-01,Beetrack,,Kowski,KOWSKI-RXTH26,ALEJANDRO OJEDA MAJIN,Jumbo,J511,JUMBO COSTANERA,2024-02-08 10:46:31,2024-02-08,Entregado,Entregado a Domicilio,Independencia,-33.413418600000000,-70.672029300000000,1,v122653017jmch-01|2024-02-08 10:47:42.509
474748,v122876301jmch-01,Beetrack,,Kowski,KOWSKI-PJLH14,GREIBY CASTILLO COLINA,Jumbo,J511,JUMBO COSTANERA,2024-02-11 09:32:29,2024-02-11,Recogido,Entregado a Domicilio,Independencia,-33.410650900000000,-70.656276400000000,1,v122876301jmch-01|2024-02-11 10:21:05.544
474749,v122770956jmch-01,Beetrack,,ZUBALE,ZUBALE-SRTP54,IVANA GERALDINE JAIMES GONZALEZ,Jumbo,J775,JUMBO ÑUÑOA,2024-02-09 16:11:01,2024-02-09,Entregado,Entregado a Domicilio,Ñuñoa,-33.462782400000000,-70.622618000000000,1,v122770956jmch-01|2024-02-09 16:12:24.78


In [39]:
#System Filter
result_df_4= result_df[result_df['orden_compra']=='v122157200jmch-01']


result_df_4

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key


**EJECUTAR 4**

In [74]:
result_df1= result_df1.rename(columns={'order_id': 'orden_compra'})



result_df1



,orden_compra,bultos
0,v102026913jmch-01,5
1,v106008740jmch-01,2
2,v111810325jmch-01,5
3,v111891091jmch-01,14
4,v112084483jmch-01,4
...,...,...
1114847,v53063398sisa-01,5
1114848,v53063399sisa-01,1
1114849,v53063401sisa-01,2
1114850,v53063402sisa-01,5


**EJECUTAR 5**

In [75]:
result_cruce = pd.merge(result_df, result_df1, on='orden_compra', how='left')

result_cruce['final_size'] = ''
result_cruce.loc[(result_cruce['bultos']>0)&(result_cruce['bultos']<=5),'final_size']='MP'
result_cruce.loc[(result_cruce['bultos']>5)&(result_cruce['bultos']<=12),'final_size']='P'
result_cruce.loc[(result_cruce['bultos']>12)&(result_cruce['bultos']<=20),'final_size']='M'
result_cruce.loc[(result_cruce['bultos']>20),'final_size']='G'

result_cruce

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size
0,v122867410jmch-01,Beetrack,,Touch,TOUCH-HVPB33,Oscar Henriquez Fierro,Jumbo,J502,JUMBO KENNEDY,2024-02-10 19:25:51,2024-02-10,Entregado,Entregado a Domicilio,Las Condes,-33.399614800000000,-70.569115500000000,1,v122867410jmch-01|2024-02-10 19:27:17.499,5.0,MP
1,v123879936jmch-01,Beetrack,,Foxer,FOXER-PSZB37,José Lara Carrizales,Jumbo,J591,JUMBO TALCA,2024-02-24 09:52:00,2024-02-24,Entregado,Entregado a Domicilio,Talca,-35.428734576275600,-71.615340886046900,1,v123879936jmch-01|2024-02-24 09:53:09.134,3.0,MP
2,v122397379jmch-01,Beetrack,,ZUBALE,ZUBALE-SBRC47,Ysmar Anselmi,Jumbo,J989,JUMBO INDEPENDENCIA,2024-02-04 15:40:13,2024-02-04,Entregado,Entregado a Domicilio,Santiago,-33.432082500000000,-70.659157700000000,1,v122397379jmch-01|2024-02-04 15:41:33.359,4.0,MP
3,v123190357jmch-01,Beetrack,,Boosmap,BOOS-UG2493,CLAUDIO ANTONIO VERGARA CHAVEZ,Jumbo,J403,JUMBO MAIPU,2024-02-15 09:54:44,2024-02-15,Entregado,Entregado a Domicilio,Maipú,-33.523223700000000,-70.772186800000000,1,v123190357jmch-01|2024-02-15 09:56:02.361,6.0,P
4,v122704818jmch-01,Beetrack,,Boosmap,BOOS-VN5369,Carolina Eugenia Aguilera Rojas,Jumbo,J843,JUMBO SAN BERNARDO,2024-02-08 18:14:09,2024-02-08,Entregado,Entregado a Domicilio,San Bernardo,-33.586949300000000,-70.697080000000000,1,v122704818jmch-01|2024-02-08 18:15:23.70,6.0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474746,v123363848jmch-01,Beetrack,,Transporte CM,TCM-YZ7148,DIEGO ANDRES VILLOUTA CERDA,Jumbo,J633,JUMBO CONCE. HUALPEN,2024-02-17 13:48:04,2024-02-17,Entregado,Entregado a Domicilio,Concepción,-36.803690000000000,-73.045779900000000,1,v123363848jmch-01|2024-02-17 13:49:15.969,8.0,P
474747,v122653017jmch-01,Beetrack,,Kowski,KOWSKI-RXTH26,ALEJANDRO OJEDA MAJIN,Jumbo,J511,JUMBO COSTANERA,2024-02-08 10:46:31,2024-02-08,Entregado,Entregado a Domicilio,Independencia,-33.413418600000000,-70.672029300000000,1,v122653017jmch-01|2024-02-08 10:47:42.509,4.0,MP
474748,v122876301jmch-01,Beetrack,,Kowski,KOWSKI-PJLH14,GREIBY CASTILLO COLINA,Jumbo,J511,JUMBO COSTANERA,2024-02-11 09:32:29,2024-02-11,Recogido,Entregado a Domicilio,Independencia,-33.410650900000000,-70.656276400000000,1,v122876301jmch-01|2024-02-11 10:21:05.544,5.0,MP
474749,v122770956jmch-01,Beetrack,,ZUBALE,ZUBALE-SRTP54,IVANA GERALDINE JAIMES GONZALEZ,Jumbo,J775,JUMBO ÑUÑOA,2024-02-09 16:11:01,2024-02-09,Entregado,Entregado a Domicilio,Ñuñoa,-33.462782400000000,-70.622618000000000,1,v122770956jmch-01|2024-02-09 16:12:24.78,14.0,M


**EJECUTAR 6**

In [76]:
import numpy as np
def get_commune(latitude, longitude):
    # Construct the API request URL
    url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"

    # Send the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Try to extract the commune (comuna) from different keys in the address details
        if 'city' in data['address']:
            return data['address']['city']
        elif 'town' in data['address']:
            return data['address']['town']
        elif 'suburb' in data['address']:
            return data['address']['suburb']
        elif 'village' in data['address']:
            return data['address']['village']
        elif 'hamlet' in data['address']:
            return data['address']['hamlet']
        else:
            return "Commune not found" 
    else:
        return "Error fetching data"  # Return error message if request was not successful

def get_communes(row):
    if pd.isna(row['comuna_destino']):  # Check if commune column is empty
        commune = get_commune(row['latitude'], row['longitude'])
        return commune
    else:
        return row['comuna_destino']
    

# Apply get_communes function to each row
result_cruce['comuna_destino'] = result_cruce.apply(get_communes, axis=1)



# Print the resulting DataFrame
result_cruce

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size
0,v122867410jmch-01,Beetrack,,Touch,TOUCH-HVPB33,Oscar Henriquez Fierro,Jumbo,J502,JUMBO KENNEDY,2024-02-10 19:25:51,2024-02-10,Entregado,Entregado a Domicilio,Las Condes,-33.399614800000000,-70.569115500000000,1,v122867410jmch-01|2024-02-10 19:27:17.499,5.0,MP
1,v123879936jmch-01,Beetrack,,Foxer,FOXER-PSZB37,José Lara Carrizales,Jumbo,J591,JUMBO TALCA,2024-02-24 09:52:00,2024-02-24,Entregado,Entregado a Domicilio,Talca,-35.428734576275600,-71.615340886046900,1,v123879936jmch-01|2024-02-24 09:53:09.134,3.0,MP
2,v122397379jmch-01,Beetrack,,ZUBALE,ZUBALE-SBRC47,Ysmar Anselmi,Jumbo,J989,JUMBO INDEPENDENCIA,2024-02-04 15:40:13,2024-02-04,Entregado,Entregado a Domicilio,Santiago,-33.432082500000000,-70.659157700000000,1,v122397379jmch-01|2024-02-04 15:41:33.359,4.0,MP
3,v123190357jmch-01,Beetrack,,Boosmap,BOOS-UG2493,CLAUDIO ANTONIO VERGARA CHAVEZ,Jumbo,J403,JUMBO MAIPU,2024-02-15 09:54:44,2024-02-15,Entregado,Entregado a Domicilio,Maipú,-33.523223700000000,-70.772186800000000,1,v123190357jmch-01|2024-02-15 09:56:02.361,6.0,P
4,v122704818jmch-01,Beetrack,,Boosmap,BOOS-VN5369,Carolina Eugenia Aguilera Rojas,Jumbo,J843,JUMBO SAN BERNARDO,2024-02-08 18:14:09,2024-02-08,Entregado,Entregado a Domicilio,San Bernardo,-33.586949300000000,-70.697080000000000,1,v122704818jmch-01|2024-02-08 18:15:23.70,6.0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474746,v123363848jmch-01,Beetrack,,Transporte CM,TCM-YZ7148,DIEGO ANDRES VILLOUTA CERDA,Jumbo,J633,JUMBO CONCE. HUALPEN,2024-02-17 13:48:04,2024-02-17,Entregado,Entregado a Domicilio,Concepción,-36.803690000000000,-73.045779900000000,1,v123363848jmch-01|2024-02-17 13:49:15.969,8.0,P
474747,v122653017jmch-01,Beetrack,,Kowski,KOWSKI-RXTH26,ALEJANDRO OJEDA MAJIN,Jumbo,J511,JUMBO COSTANERA,2024-02-08 10:46:31,2024-02-08,Entregado,Entregado a Domicilio,Independencia,-33.413418600000000,-70.672029300000000,1,v122653017jmch-01|2024-02-08 10:47:42.509,4.0,MP
474748,v122876301jmch-01,Beetrack,,Kowski,KOWSKI-PJLH14,GREIBY CASTILLO COLINA,Jumbo,J511,JUMBO COSTANERA,2024-02-11 09:32:29,2024-02-11,Recogido,Entregado a Domicilio,Independencia,-33.410650900000000,-70.656276400000000,1,v122876301jmch-01|2024-02-11 10:21:05.544,5.0,MP
474749,v122770956jmch-01,Beetrack,,ZUBALE,ZUBALE-SRTP54,IVANA GERALDINE JAIMES GONZALEZ,Jumbo,J775,JUMBO ÑUÑOA,2024-02-09 16:11:01,2024-02-09,Entregado,Entregado a Domicilio,Ñuñoa,-33.462782400000000,-70.622618000000000,1,v122770956jmch-01|2024-02-09 16:12:24.78,14.0,M


**EJECUTAR 7**

In [77]:
result_cruce['provider_key'] = result_cruce['codigo_tienda']+"|"+result_cruce['proveedor']+'|'+result_cruce['comuna_destino']+'|'+result_cruce['final_size']

result_cruce

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,...,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size,provider_key
0,v122867410jmch-01,Beetrack,,Touch,TOUCH-HVPB33,Oscar Henriquez Fierro,Jumbo,J502,JUMBO KENNEDY,2024-02-10 19:25:51,...,Entregado,Entregado a Domicilio,Las Condes,-33.399614800000000,-70.569115500000000,1,v122867410jmch-01|2024-02-10 19:27:17.499,5.0,MP,J502|Touch|Las Condes|MP
1,v123879936jmch-01,Beetrack,,Foxer,FOXER-PSZB37,José Lara Carrizales,Jumbo,J591,JUMBO TALCA,2024-02-24 09:52:00,...,Entregado,Entregado a Domicilio,Talca,-35.428734576275600,-71.615340886046900,1,v123879936jmch-01|2024-02-24 09:53:09.134,3.0,MP,J591|Foxer|Talca|MP
2,v122397379jmch-01,Beetrack,,ZUBALE,ZUBALE-SBRC47,Ysmar Anselmi,Jumbo,J989,JUMBO INDEPENDENCIA,2024-02-04 15:40:13,...,Entregado,Entregado a Domicilio,Santiago,-33.432082500000000,-70.659157700000000,1,v122397379jmch-01|2024-02-04 15:41:33.359,4.0,MP,J989|ZUBALE|Santiago|MP
3,v123190357jmch-01,Beetrack,,Boosmap,BOOS-UG2493,CLAUDIO ANTONIO VERGARA CHAVEZ,Jumbo,J403,JUMBO MAIPU,2024-02-15 09:54:44,...,Entregado,Entregado a Domicilio,Maipú,-33.523223700000000,-70.772186800000000,1,v123190357jmch-01|2024-02-15 09:56:02.361,6.0,P,J403|Boosmap|Maipú|P
4,v122704818jmch-01,Beetrack,,Boosmap,BOOS-VN5369,Carolina Eugenia Aguilera Rojas,Jumbo,J843,JUMBO SAN BERNARDO,2024-02-08 18:14:09,...,Entregado,Entregado a Domicilio,San Bernardo,-33.586949300000000,-70.697080000000000,1,v122704818jmch-01|2024-02-08 18:15:23.70,6.0,P,J843|Boosmap|San Bernardo|P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474746,v123363848jmch-01,Beetrack,,Transporte CM,TCM-YZ7148,DIEGO ANDRES VILLOUTA CERDA,Jumbo,J633,JUMBO CONCE. HUALPEN,2024-02-17 13:48:04,...,Entregado,Entregado a Domicilio,Concepción,-36.803690000000000,-73.045779900000000,1,v123363848jmch-01|2024-02-17 13:49:15.969,8.0,P,J633|Transporte CM|Concepción|P
474747,v122653017jmch-01,Beetrack,,Kowski,KOWSKI-RXTH26,ALEJANDRO OJEDA MAJIN,Jumbo,J511,JUMBO COSTANERA,2024-02-08 10:46:31,...,Entregado,Entregado a Domicilio,Independencia,-33.413418600000000,-70.672029300000000,1,v122653017jmch-01|2024-02-08 10:47:42.509,4.0,MP,J511|Kowski|Independencia|MP
474748,v122876301jmch-01,Beetrack,,Kowski,KOWSKI-PJLH14,GREIBY CASTILLO COLINA,Jumbo,J511,JUMBO COSTANERA,2024-02-11 09:32:29,...,Recogido,Entregado a Domicilio,Independencia,-33.410650900000000,-70.656276400000000,1,v122876301jmch-01|2024-02-11 10:21:05.544,5.0,MP,J511|Kowski|Independencia|MP
474749,v122770956jmch-01,Beetrack,,ZUBALE,ZUBALE-SRTP54,IVANA GERALDINE JAIMES GONZALEZ,Jumbo,J775,JUMBO ÑUÑOA,2024-02-09 16:11:01,...,Entregado,Entregado a Domicilio,Ñuñoa,-33.462782400000000,-70.622618000000000,1,v122770956jmch-01|2024-02-09 16:12:24.78,14.0,M,J775|ZUBALE|Ñuñoa|M


In [19]:
result_cruce_2=result_cruce[result_cruce['orden_compra']=='v120919018jmch-01']

result_cruce_2

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,...,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size,provider_key
90067,v120919018jmch-01,Beetrack,,ZUBALE,ZUBALE-GXRW35,Jocelyn Cristina Reyes,Jumbo,J504,JUMBO RANCAGUA,2024-02-03 21:46:35,...,Entregado,Entregado a Domicilio,Las Cabras,-34.158049468051400,-71.470549308961600,0,v120919018jmch-01|2024-02-03 21:48:16.902,13.0,M,J504|ZUBALE|Las Cabras|M


**EJECUTAR 8**

In [78]:
#Open Pricing


pricing_excel_file_path = "/Users/eliassaleh/Desktop/Facturacion/Tarifario.xlsx"

df_pricing = pd.read_excel(pricing_excel_file_path)

df_pricing


,Business,Store_ID,Provider,Destination,Size,Cost,HD
0,Santa Isabel,N841,Transporte Campillay,Copiapó,MP,4900,NO
1,Santa Isabel,N841,Transporte Campillay,Caldera,MP,4900,NO
2,Santa Isabel,N841,Transporte Campillay,Tierra Amarilla,MP,4900,NO
3,Santa Isabel,N841,Transporte Campillay,Huasco,MP,4900,NO
4,Santa Isabel,N841,Transporte Campillay,Vallenar,MP,4900,NO
...,...,...,...,...,...,...,...
19736,Jumbo,J532,ZUBALE,Quillón,P,16331,NO
19737,Jumbo,J532,ZUBALE,Quillón,M,17147,NO
19738,Jumbo,J532,ZUBALE,Quillón,G,21169,NO
19739,Jumbo,J532,ZUBALE,Quillón,Extra G,23909,NO


**EJECUTAR 9**

In [79]:
df_pricing['provider_key'] = df_pricing['Store_ID'].astype(str)+"|"+df_pricing['Provider']+'|'+df_pricing['Destination']+'|'+df_pricing['Size']




df_pricing

,Business,Store_ID,Provider,Destination,Size,Cost,HD,provider_key
0,Santa Isabel,N841,Transporte Campillay,Copiapó,MP,4900,NO,N841|Transporte Campillay|Copiapó|MP
1,Santa Isabel,N841,Transporte Campillay,Caldera,MP,4900,NO,N841|Transporte Campillay|Caldera|MP
2,Santa Isabel,N841,Transporte Campillay,Tierra Amarilla,MP,4900,NO,N841|Transporte Campillay|Tierra Amarilla|MP
3,Santa Isabel,N841,Transporte Campillay,Huasco,MP,4900,NO,N841|Transporte Campillay|Huasco|MP
4,Santa Isabel,N841,Transporte Campillay,Vallenar,MP,4900,NO,N841|Transporte Campillay|Vallenar|MP
...,...,...,...,...,...,...,...,...
19736,Jumbo,J532,ZUBALE,Quillón,P,16331,NO,J532|ZUBALE|Quillón|P
19737,Jumbo,J532,ZUBALE,Quillón,M,17147,NO,J532|ZUBALE|Quillón|M
19738,Jumbo,J532,ZUBALE,Quillón,G,21169,NO,J532|ZUBALE|Quillón|G
19739,Jumbo,J532,ZUBALE,Quillón,Extra G,23909,NO,J532|ZUBALE|Quillón|Extra G


In [22]:
df_pricing1=df_pricing[df_pricing['provider_key']=='J504|ZUBALE|Las Cabras|M']

df_pricing1

,Business,Store_ID,Provider,Destination,Size,Cost,HD,provider_key
17236,Jumbo,J504,ZUBALE,Las Cabras,M,25903,NO,J504|ZUBALE|Las Cabras|M


**EJECUTAR 10**

In [80]:
result_pricing = pd.merge(result_cruce, df_pricing, on='provider_key', how= 'left')


result_pricing = result_pricing[['orden_compra', 'sistema', 'codigo_conductor','proveedor','patente','nombre_driver','negocio','codigo_tienda','nombre_tienda','fecha_evento','fecha_compromiso','nombre_estado','nombre_sub_estado','comuna_destino','on_time','bultos','final_size','provider_key','Cost']]

result_pricing['final_cost'] = result_pricing.apply(lambda row: row['Cost'] * 0.5 if row['nombre_estado'] == 'Entregado Parcial' else (row['Cost'] * 0.5 if row['nombre_estado'] == 'No Entregado' else row['Cost']), axis=1)
result_pricing

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost
0,v122867410jmch-01,Beetrack,,Touch,TOUCH-HVPB33,Oscar Henriquez Fierro,Jumbo,J502,JUMBO KENNEDY,2024-02-10 19:25:51,2024-02-10,Entregado,Entregado a Domicilio,Las Condes,1,5.0,MP,J502|Touch|Las Condes|MP,3825.0,3825.0
1,v123879936jmch-01,Beetrack,,Foxer,FOXER-PSZB37,José Lara Carrizales,Jumbo,J591,JUMBO TALCA,2024-02-24 09:52:00,2024-02-24,Entregado,Entregado a Domicilio,Talca,1,3.0,MP,J591|Foxer|Talca|MP,4561.0,4561.0
2,v122397379jmch-01,Beetrack,,ZUBALE,ZUBALE-SBRC47,Ysmar Anselmi,Jumbo,J989,JUMBO INDEPENDENCIA,2024-02-04 15:40:13,2024-02-04,Entregado,Entregado a Domicilio,Santiago,1,4.0,MP,J989|ZUBALE|Santiago|MP,NaN,NaN
3,v123190357jmch-01,Beetrack,,Boosmap,BOOS-UG2493,CLAUDIO ANTONIO VERGARA CHAVEZ,Jumbo,J403,JUMBO MAIPU,2024-02-15 09:54:44,2024-02-15,Entregado,Entregado a Domicilio,Maipú,1,6.0,P,J403|Boosmap|Maipú|P,5390.0,5390.0
4,v122704818jmch-01,Beetrack,,Boosmap,BOOS-VN5369,Carolina Eugenia Aguilera Rojas,Jumbo,J843,JUMBO SAN BERNARDO,2024-02-08 18:14:09,2024-02-08,Entregado,Entregado a Domicilio,San Bernardo,1,6.0,P,J843|Boosmap|San Bernardo|P,5585.0,5585.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474746,v123363848jmch-01,Beetrack,,Transporte CM,TCM-YZ7148,DIEGO ANDRES VILLOUTA CERDA,Jumbo,J633,JUMBO CONCE. HUALPEN,2024-02-17 13:48:04,2024-02-17,Entregado,Entregado a Domicilio,Concepción,1,8.0,P,J633|Transporte CM|Concepción|P,5250.0,5250.0
474747,v122653017jmch-01,Beetrack,,Kowski,KOWSKI-RXTH26,ALEJANDRO OJEDA MAJIN,Jumbo,J511,JUMBO COSTANERA,2024-02-08 10:46:31,2024-02-08,Entregado,Entregado a Domicilio,Independencia,1,4.0,MP,J511|Kowski|Independencia|MP,4800.0,4800.0
474748,v122876301jmch-01,Beetrack,,Kowski,KOWSKI-PJLH14,GREIBY CASTILLO COLINA,Jumbo,J511,JUMBO COSTANERA,2024-02-11 09:32:29,2024-02-11,Recogido,Entregado a Domicilio,Independencia,1,5.0,MP,J511|Kowski|Independencia|MP,4800.0,4800.0
474749,v122770956jmch-01,Beetrack,,ZUBALE,ZUBALE-SRTP54,IVANA GERALDINE JAIMES GONZALEZ,Jumbo,J775,JUMBO ÑUÑOA,2024-02-09 16:11:01,2024-02-09,Entregado,Entregado a Domicilio,Ñuñoa,1,14.0,M,J775|ZUBALE|Ñuñoa|M,5180.0,5180.0


In [340]:
result_pricing_2=result_pricing[result_pricing['codigo_tienda']=='J514']
result_pricing_2

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost
46199,v122433741jmch-01,Beetrack,,Boosmap,BOOS-FJXR36,Miguel Aguilera,Jumbo,J514,JUMBO LA DEHESA,2024-02-05 10:25:04,2024-02-05,Entregado,Entregado a Domicilio,None,1,3.0,MP,NaN,10240,10240.0
46200,v122433741jmch-01,Beetrack,,Boosmap,BOOS-FJXR36,Miguel Aguilera,Jumbo,J514,JUMBO LA DEHESA,2024-02-05 10:25:04,2024-02-05,Entregado,Entregado a Domicilio,None,1,3.0,MP,NaN,6570,6570.0
46201,v122433741jmch-01,Beetrack,,Boosmap,BOOS-FJXR36,Miguel Aguilera,Jumbo,J514,JUMBO LA DEHESA,2024-02-05 10:25:04,2024-02-05,Entregado,Entregado a Domicilio,None,1,3.0,MP,NaN,6570,6570.0
46202,v122433741jmch-01,Beetrack,,Boosmap,BOOS-FJXR36,Miguel Aguilera,Jumbo,J514,JUMBO LA DEHESA,2024-02-05 10:25:04,2024-02-05,Entregado,Entregado a Domicilio,None,1,3.0,MP,NaN,5390,5390.0
46203,v122433741jmch-01,Beetrack,,Boosmap,BOOS-FJXR36,Miguel Aguilera,Jumbo,J514,JUMBO LA DEHESA,2024-02-05 10:25:04,2024-02-05,Entregado,Entregado a Domicilio,None,1,3.0,MP,NaN,15810,15810.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241284,v122111526jmch-01,Beetrack,,Touch,TOUCH-ZW2711,Melisa del Carmen Álvarez Mondaca,Jumbo,J514,JUMBO LA DEHESA,2024-02-01 11:16:22,2024-02-01,Entregado,Entregado a Domicilio,Las Condes,1,3.0,MP,J514|Touch|Las Condes|MP,3927,3927.0
241285,v122074783jmch-01,Beetrack,,Touch,TOUCH-SFWB69,Diego Araque,Jumbo,J514,JUMBO LA DEHESA,2024-02-01 09:04:20,2024-02-01,Entregado,Entregado a Domicilio,Las Condes,1,4.0,MP,J514|Touch|Las Condes|MP,3927,3927.0
242048,v122104948jmch-01,Beetrack,,Touch,TOUCH-KDDF86,Juan Camilo Soto Cifuentes,Jumbo,J514,JUMBO LA DEHESA,2024-02-01 09:32:02,2024-02-01,Entregado,Entregado a Domicilio,Santiago,1,3.0,MP,J514|Touch|Santiago|MP,4398,4398.0
242104,v122117192jmch-01,Beetrack,,Touch,TOUCH-PJSH57,Natalia Carvajal,Jumbo,J514,JUMBO LA DEHESA,2024-02-01 12:14:52,2024-02-01,Entregado,Entregado a Domicilio,Santiago,1,18.0,M,J514|Touch|Santiago|M,5561,5561.0


In [25]:
result_pricing_3=result_pricing[result_pricing['orden_compra']=='v122433741jmch-01']
result_pricing_3

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost
497,v122433741jmch-01,Beetrack,,Boosmap,BOOS-FJXR36,Miguel Aguilera,Jumbo,J514,JUMBO LA DEHESA,2024-02-05 10:25:04,2024-02-05,Entregado,Entregado a Domicilio,Colina,1,3.0,MP,J514|Boosmap|Colina|MP,NaN,NaN


In [48]:
result_pricing_5=result_pricing
result_pricing_5['commune_status'] = result_pricing_5['comuna_destino'].apply(lambda x: 'Empty' if pd.isna(x) else 'Not Empty')

result_pricing_5[result_pricing_5['commune_status']=='Empty']

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,...,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost,commune_status


**EJECUTAR 11**

In [81]:
def format_clp(x):
    return '{:,.0f} CLP'.format(x)

def format_thousands(x):
    return '{:,.0f}'.format(x).replace(',', '.')


pivot_table = pd.pivot_table(result_pricing, values='final_cost', index=['codigo_tienda', 'nombre_tienda', 'proveedor'], aggfunc= {'final_cost': ['sum', 'count']})

# Reset index to flatten the pivot table
pivot_table = pivot_table.reset_index()

pivot_table['sum'] = pivot_table['sum'].apply(format_thousands)
pivot_table['count'] = pivot_table['count'].apply(format_thousands)
#pivot_table['final_cost'] = pivot_table['final_cost'].apply(format_clp)



pivot_table


,codigo_tienda,nombre_tienda,proveedor,count,sum
0,J403,JUMBO MAIPU,Boosmap,20.829,118.056.423
1,J403,JUMBO MAIPU,Boosmap Chile,7.801,44.123.881
2,J410,JUMBO FLORIDA,Kowski Chile,33.712,169.361.090
3,J414,DARKSTORE LA DEHESA,Boosmap,0,0
4,J502,JUMBO KENNEDY,Beat,0,0
5,J502,JUMBO KENNEDY,Boosmap,0,0
6,J502,JUMBO KENNEDY,Casos Touch,0,0
7,J502,JUMBO KENNEDY,Touch,41.184,159.241.934
8,J504,JUMBO RANCAGUA,ZUBALE,6.036,35.110.532
9,J504,JUMBO RANCAGUA,Zubale,3.035,16.636.563


**EJECUTAR 12**

In [82]:
pivot_table2 = pd.pivot_table(result_cruce, values='orden_compra', index=['codigo_tienda', 'nombre_tienda'], aggfunc= {'orden_compra': ['count']})

# Reset index to flatten the pivot table
pivot_table2 = pivot_table2.reset_index()

pivot_table2['count'] = pivot_table2['count'].apply(format_thousands)
#pivot_table['final_cost'] = pivot_table['final_cost'].apply(format_clp)

pivot_table2

,codigo_tienda,nombre_tienda,count
0,J403,JUMBO MAIPU,29.102
1,J410,JUMBO FLORIDA,33.736
2,J414,DARKSTORE LA DEHESA,28.508
3,J502,JUMBO KENNEDY,41.872
4,J504,JUMBO RANCAGUA,9.087
5,J506,JUMBO TEMUCO,5.708
6,J507,JUMBO PTO. MONTT,4.326
7,J508,JUMBO VIÑA,31.575
8,J511,JUMBO COSTANERA,54.419
9,J512,JUMBO LA REINA,45.710


**EJECUTAR 13**

In [83]:
#Create Excel File

excel_file_path = f'/Users/eliassaleh/Desktop/Facturacion/Exportables/Exportable_Delivery_{negocio}_{fecha_compromiso_inicial[:10]}_{fecha_compromiso_final[:10]}.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    pivot_table.to_excel(writer,sheet_name ='Consolidado_Tarifario',index=False, engine='xlsxwriter')
    pivot_table2.to_excel(writer,sheet_name ='Consolidado_Completo',index=False, engine='xlsxwriter')
    result_pricing.to_excel(writer,sheet_name ='BDD_Tarifario',index=False, engine='xlsxwriter')
   
    





**NO USAR**

In [29]:
#Create Excel File (Backup with inner join between cruce and pricing)

excel_file_path = f'/Users/eliassaleh/Desktop/Facturacion/Exportables/Exportable_Delivery_{negocio}_{fecha_compromiso_inicial[:10]}_{fecha_compromiso_final[:10]}.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    pivot_table.to_excel(writer,sheet_name ='Consolidado_Tarifario',index=False, engine='xlsxwriter')
    result_pricing.to_excel(writer,sheet_name ='BDD_Tarifario',index=False, engine='xlsxwriter')
    df_pricing.to_excel(writer,sheet_name ='Tarifario',index=False, engine='xlsxwriter')
    pivot_table2.to_excel(writer,sheet_name ='Consolidado_Completo',index=False, engine='xlsxwriter')
    result_cruce.to_excel(writer,sheet_name ='BDD_Completa',index=False, engine='xlsxwriter')

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/2e/7d/2259318c202f3d17f3fe6438149b3b9e706d1070fe3fcbb28049730bb25c/charset_normalizer-3.3.2-cp312-cp312-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/c2/e7/a82b05cf63a603df6e68d59ae6a68bf5064484a0718ea5033660af4b54a9/idna-3.6-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/88/75/311454fd3317aefe18415f04568edc20218453b709c63c58b9292c71be17/urllib3-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e9

In [354]:
import requests

def get_commune(latitude, longitude):
    # Construct the API request URL
    url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"




    # Send the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Try to extract the commune (comuna) from different keys in the address details
        if 'city' in data['address']:
            return data['address']['city']
        elif 'town' in data['address']:
            return data['address']['town']
        elif 'village' in data['address']:
            return data['address']['village']
        elif 'hamlet' in data['address']:
            return data['address']['hamlet']
        else:
            return "Commune not found" 
    else:
        return "Error fetching data"  # Return error message if request was not successful

# Example usage
latitude = -34.158049468051400	
longitude = -71.470549308961600



commune = get_commune(latitude, longitude)
print("Commune:", commune)







Commune: Las Cabras


In [307]:
import numpy as np
def get_commune(latitude, longitude):
    # Construct the API request URL
    url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"

    # Send the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Print API response for debugging
        print("API Response:", data)
        
        # Try to extract the commune (comuna) from different keys in the address details
        if 'city' in data['address']:
            return data['address']['city']
        elif 'town' in data['address']:
            return data['address']['town']
        elif 'suburb' in data['address']:
            return data['address']['suburb']
        elif 'village' in data['address']:
            return data['address']['village']
        elif 'hamlet' in data['address']:
            return data['address']['hamlet']
        else:
            print("Address details:", data['address'])  # Print out address details for debugging
            return "Commune not found" 
    else:
        print("Error fetching data")  # Print error message for debugging
        return "Error fetching data"  # Return error message if request was not successful

def get_communes(row):
    if pd.isna(row['commune']):  # Check if commune column is empty
        commune = get_commune(row['latitude'], row['longitude'])
        return commune
    else:
        return row['commune']
    

# Example DataFrame with latitude and longitude columns
data = {
    'latitude': [-39.07890413, -33.4489, -33.4691],
    'longitude': [-71.97376793, -70.6693, -70.642],
    'commune': [np.nan, 'Santiago', np.nan]
}
df_coord = pd.DataFrame(data)


# Apply get_communes function to each row
df_coord['commune'] = df_coord.apply(get_communes, axis=1)



# Print the resulting DataFrame
print("Resulting DataFrame:")
df_coord



#url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"

    # Send the request
#response = requests.get(url)
    

#data = response.json()

#latitude= -39.078904
#longitude=- 71.973768

#print("API Response:", data)
#commune = get_commune(latitude, longitude)
#print("Commune:", commune)

API Response: {'place_id': 33550931, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 6009239545, 'lat': '-39.0641046', 'lon': '-71.97025', 'category': 'place', 'type': 'isolated_dwelling', 'place_rank': 22, 'importance': 0.2000099999999999, 'addresstype': 'isolated_dwelling', 'name': 'Nancahue', 'display_name': 'Nancahue, Cunco, Provincia de Cautín, Región de la Araucanía, Chile', 'address': {'isolated_dwelling': 'Nancahue', 'town': 'Cunco', 'county': 'Provincia de Cautín', 'state': 'Región de la Araucanía', 'ISO3166-2-lvl4': 'CL-AR', 'country': 'Chile', 'country_code': 'cl'}, 'boundingbox': ['-39.0641546', '-39.0640546', '-71.9703000', '-71.9702000']}
API Response: {'place_id': 61385429, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 110572520, 'lat': '-33.46918361818182', 'lon': '-70.64198729999998', 'category': 'place', 'type': 'house', 'place_rank': 30

,latitude,longitude,commune
0,-39.078904,-71.973768,Cunco
1,-33.448900,-70.669300,Santiago
2,-33.469100,-70.642000,Santiago
